# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [25]:
# 전체 데이터 수
all = len(pd_data)

# 각 class의 확률값
P_class = [(value/all)**2 for key, value in pd_data['class_buys_computer'].value_counts().items()]
print(P_class)

# 누적값을 받기 위해 reduce 함수 사용
from functools import reduce
gini = reduce(lambda x, y: x - y, P_class, 1)
print(gini)

[0.41326530612244905, 0.12755102040816327]
0.4591836734693877


In [211]:
def get_gini(df, label):
    all = len(df)
    P_class = [(value/all)**2 for key, value in df[label].value_counts().items()]
    gini = reduce(lambda x, y: x - y, P_class, 1)
    return gini

In [198]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [78]:
from itertools import combinations

uniques = list(pd_data['age'].unique())
print(uniques)

result = []

for i in range(len(uniques)-1):
    for j in combinations(uniques, i+1):
        result.append(list(j))
        
        
print(result)

['youth', 'middle_aged', 'senior']
[['youth'], ['middle_aged'], ['senior'], ['youth', 'middle_aged'], ['youth', 'senior'], ['middle_aged', 'senior']]


In [73]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    result = []
    
    for i in range(len(uniques)-1):
        for j in combinations(uniques, i+1):
            result.append(list(j))
    
    return result

In [74]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [218]:
result = {}
binary_split = get_binary_split(pd_data, 'income')
n = len(pd_data)

for binary in binary_split:
    num_series = pd_data['income'].map(lambda x: x in binary)
    gini = (sum(num_series) / n) * (get_gini(pd_data[num_series], 'class_buys_computer')) + ((n-sum(num_series))/n) * get_gini(pd_data[num_series == False], 'class_buys_computer')

    if len(binary) == 1:
        result[binary[0]] = round(gini, 4)
    else:
        result[reduce(lambda x, y: x + '_' + y, binary, '')[1:]] = round(gini,3)
result    

{'high': 0.4429,
 'medium': 0.4583,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.458,
 'medium_low': 0.443}

In [222]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)

    for binary in binary_split:
        num_series = df[attribute].map(lambda x: x in binary)
        gini = (sum(num_series) / n) * (get_gini(df[num_series], label)) + ((n-sum(num_series))/n) * get_gini(df[num_series == False], 'class_buys_computer')
        if len(binary) == 1:
            result[binary[0]] = round(gini, 4)
        else:
            result[reduce(lambda x, y: x + '_' + y, binary, '')[1:]] = round(gini,4)
    
    return result

In [223]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.4429,
 'medium': 0.4583,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583,
 'medium_low': 0.4429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [224]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.4429)

In [225]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

'high'

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [226]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    score = min(get_attribute_gini_index(pd_data, feature, 'class_buys_computer').items())[1]
    
    print(f"Minimum Gini index of {feature} : {score}")

Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [227]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.3937,
 'middle_aged': 0.3571,
 'senior': 0.4571,
 'youth_middle_aged': 0.4571,
 'youth_senior': 0.3571,
 'middle_aged_senior': 0.3937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [272]:
from math import log2

def getEntropy(df, feature) :
    
    
    """
    수식을 참고하여,
    데이터프레임 df에서 특정 feature에 대해 
    엔트로피를 구하는 함수를 작성해주세요.
    """
    
    all_num = len(df)
    return reduce(lambda x,y: x - (y[1]/all_num * log2(y[1]/all_num)), list(df['class_buys_computer'].value_counts().items()), 0)

In [273]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [310]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
        
    result = {}
    fea_li = list(df.columns)
    fea_li.remove(feature)
    
    target = getEntropy(df, feature)
    all_num = len(df)
    
    for fea in fea_li:
        elements = df[fea].value_counts().items()
        score = 0
        for element, value in elements:
            num_series = df[fea].map(lambda x: x in element)
            ent = getEntropy(df[num_series], feature)
            score += ent * (len(df[num_series]) / all_num)
            
        final_score = target - score
        result[fea] = final_score
        
            
    
    return result

getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}

In [21]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443922,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}